In [ ]:
# External packages

import intake
from easygems import healpix as egh

import numpy as np

import matplotlib.pyplot as plt

import calendar

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Functions from our repo

import tools.utils as sc

In [ ]:
# Time period
time = ('2020-04-01','2020-04-30')
# time = ('2020-08-01','2020-08-31')

# Region
map_domain = sc.domains10x10['namibian']

In [ ]:
# Dataset specification

hknode = 'EU'

simulations = {
    'IFS': {
        'id':    'ifs_tco3999-ng5_rcbmf_cf',
        'opt':   {'zoom':11, 'time':'PT1H'},
        'rnm':   {'level':'pressure', 'value':'cell', 'clwvi':'lwp'},
        'vunits': 'hPa'
    },
    'ICON': {
        'id':    'icon_d3hp003',
        'opt':   {'zoom':11, 'time':'PT6H', 'time_method':'inst'},
        'rnm':   {'qall':'lwc'}
    },
    'NICAM': {
        'id':    'nicam_gl11',
        'opt':   {'zoom': 9, 'time':'PT6H'},
        'rnm':   {'lev':'pressure', 'qall':'lwc'}
    },
    'UM': {
        'id':    'um_glm_n2560_RAL3p3',
        'opt':   {'zoom':10, 'time':'PT3H'},
        'rnm':   {'clw':'lwc'}
    }
}

In [ ]:
# Load datasets

cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")[hknode]

ds = {}

for name, params in simulations.items():
    ds[name] = cat[params['id']](**params['opt']).to_dask() \
        .rename(params['rnm']).pipe(egh.attach_coords)

    if 'pressure' in ds[name] and 'units' not in ds[name]['pressure'].attrs:
        ds[name]['pressure'].attrs['units'] = params['vunits']
    
    print(name)

In [ ]:
%%time

# Select time period and region

for name in ds.keys():
    cells = egh.isel_extent(ds[name],map_domain)
    ds[name] = ds[name].sel(time=slice(*time)).isel({'cell':cells})

In [ ]:
%%time

# Derive lwp and wamax

for name in ds.keys():

    if 'lwp' not in ds[name] and 'lwc' in ds[name]:
        ds[name]['lwp'] = sc.integrate_wrt_pressure(ds[name]['lwc'])
    ds[name]['lwp'].attrs = {'name':'LWP','units':'kg/m^2'}
    print(name+' LWP integration')
    
    if 'wa' in ds[name]:
        ds[name]['wamax'] = sc.reduce_below(ds[name]['wa'],900e2,np.max)
        ds[name]['wamax'].attrs = {'name':'Max w <900hPa','units':'m/s'}
        print(name+' wa max <900hPa')

In [ ]:
%%time

# Calculate simple stats

for name in ds.keys():
    for var in ['lwp','wamax']:
        ds[name] = sc.basic_stats(ds[name],var)
        print(name+' '+var)

In [ ]:
# Read EarthCare file

ecfile = './stats_earthcare_April2025.csv'

month = calendar.month_name[np.datetime64(time[0]).item().month]
if month=='April':
    ds['EC'] = sc.read_earthcare_csv(ecfile)

In [ ]:
# Plot timeseries

plot_path = f"./figures/"

stats = ['_mean','_std','_skw','_hom']
variables = ['lwp','wamax']
labels = ['LWP [kg/m2]','Max w <900hPa [m/s]']

for var, lab in zip(variables,labels):
    Npanel = len(stats)
    fig, axs = plt.subplots(Npanel,1, figsize=(12,3*Npanel),
                            sharex=True, constrained_layout=True)
    
    for ax, stat in zip(axs,stats):
        leg = []
        for name in ds.keys():
            if name=='EC':
                if var+stat in ds[name]:
                    ax.plot(ds[name].time, ds[name][var+stat], marker='o', color='black')
                    leg.append(name)                
            else:
                ax.plot(ds[name].time, ds[name][var+stat])
                leg.append(name)
        ax.set_ylabel(lab+stat.replace('_',' '))
        ax.grid()
        ax.autoscale(tight=True)
        
    fig.legend(labels=leg,bbox_to_anchor=(0.5,0),loc='upper center',ncol=len(ds))
    
    plt.savefig(plot_path+'stat_timeseries_'+var+'_'+month,bbox_inches='tight',dpi=300)

In [ ]:
# Plot histograms

plot_path = f"./figures/"

variables = ['lwp','wamax']
labels = ['LWP [kg/m2]','Max w <900hPa [m/s]']
binedges = [np.arange(0,0.4,0.01), np.arange(0,0.5,0.01)]

for var, be, lab in zip(variables,binedges,labels):
    
    plt.figure()
    for name in simulations.keys():
        plt.hist( np.ravel(ds[name][var]), bins=be, density=True, histtype='step', label=name)
    plt.legend()
    plt.grid()
    plt.ylabel('PDF')
    plt.xlabel(lab)
    plt.yscale('log')
    plt.ylim((1e-1,1e2))

    plt.savefig(plot_path+'stat_histograms_'+var+'_'+month,bbox_inches='tight',dpi=300)